In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 17)

### Tables in the process

In [2]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
    
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'SHARES':'{:,}',    
    'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','DIVIDEND':'{:.4f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
}

In [ ]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

### In case of sale, retreive buy id from sells record (buy_id)

In [ ]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

In [ ]:
transaction, name, qty, sell_price, buy_price

### In case of buy, retreive buy id from latest buy record (id)

In [3]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,PTTGC,1101,95,2022-01-17,1600,59.25,196.24,13.74,Active,95009.98,DTD,None


In [4]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'PTTGC', 1600, 59.25)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

In [ ]:
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)

In [ ]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)

### If shares become zero and no longer wants to keep dividend record

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

In [ ]:
sell_target = 150.5 # must calculate manually beforehand
#reason = "15pct"

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
sell_target = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_target, name)
print(sqlUpd)

In [ ]:
rp = conlite.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [5]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy['volbuy'] = buy['volbuy'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'PTTGC'



,name,date,volbuy,price,active,dividend,period,grade
0,PTTGC,2021-03-17,4400,66.75,1,3.0,1,A2


### In case there is the same stock name in port

In [6]:
new_cost = 64.75 # new average cost must be calculated manually beforehand

sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 1600, price = 64.75
WHERE name = 'PTTGC'



In [7]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [8]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy['volbuy'] = buy['volbuy'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'PTTGC'



,name,date,volbuy,price,active,dividend,period,grade
0,PTTGC,2021-03-17,6000,64.75,1,3.0,1,A2


### In case of totally new stock

In [ ]:
sqlIns = """
INSERT INTO buy 
VALUES('LPF','2022-01-14',10000,14,0,0,1,0,'4','A3')
"""
print(sqlIns)

In [ ]:
#rp = const.execute(sqlIns)
rp.rowcount

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [9]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'PTTGC' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,PTTGC,1.0000,0.0000,2.0000,0.0000,3.0000,"4,400",2022-02-25,2021-09-22,None,0


In [10]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 1600
WHERE name = 'PTTGC'



In [11]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'PTTGC' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,PTTGC,1.0000,0.0000,2.0000,0.0000,3.0000,"6,000",2022-02-25,2021-09-22,None,0


### End of Update dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [13]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)


SELECT * 
FROM stocks 
WHERE name = 'PTTGC'



In [14]:
stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,135,PTTGC,70.00,54.25,B,59.25,0.00,"1,127.95",1.23,66.75,"1,600",-11,9,"4,400",3,12,RD10pct,SET50


In [15]:
new_cost = 64.75
buy_target = 57.25
sell_target = 0
status = 'B'
reason = 'RD10pct'
new_buy_qty = 1200
new_cost, buy_target, status, reason, new_buy_qty

(64.75, 57.25, 'B', 'RD10pct', 1200)

In [16]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, status, reason, new_buy_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 1600, 
cost = 64.75, sell_target = 57.25, status = 'B', 
reason = 'RD10pct', qty = 1200
WHERE name = 'PTTGC'



In [17]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [18]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'PTTGC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,135,PTTGC,70.00,54.25,B,59.25,57.25,"1,127.95",1.23,64.75,"1,200",-11,9,"6,000",3,12,RD10pct,SET50


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process